In [1]:
DRAFT = True
import itertools
import logging
import warnings
import numpy as np
print(np.version.version)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", "plotnine\..*")

import cmdstanpy as csp
csp.utils.get_logger().setLevel(logging.ERROR)


import statistics as stat
import pandas as pd
import plotnine as pn
import patchworklib as pw

1.23.1
No module named 'seaborn._core'


<Figure size 72x72 with 0 Axes>

In [2]:
f = open('binomial-rng.stan', 'w')
f.write("""

data {
    int<lower=0> N;
    real<lower=0, upper=1> theta;
}
generated quantities{
    int<lower=0, upper=N> y = binomial_rng(N, theta);
}

""")
f.close()

In [5]:
N = 100
theta = 0.3
data = {'N': N, 'theta': theta}
model = csp.CmdStanModel(stan_file = 'binomial-rng.stan')
sample = model.sample(data = data, seed = 123, chains = 1, 
                      iter_sampling = 10, iter_warmup = 0,
                     show_progress = True, show_console = True)

12:45:12 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


Chain [1] 


RuntimeError: Error during sampling:

Command and output files:
RunSet: chains=1, chain_ids=[1], num_processes=1
 cmd (chain 1):
	['C:\\Users\\manny\\Documents\\Research\\Flatiron\\PPL-practice\\Stan\\binomial-rng.exe', 'id=1', 'random', 'seed=123', 'data', 'file=C:\\Users\\manny\\AppData\\Local\\Temp\\tmpc5xbustq\\hkrzyc7q.json', 'output', 'file=C:\\Users\\manny\\AppData\\Local\\Temp\\tmpc5xbustq\\binomial-rng62poy055\\binomial-rng-20230605124511.csv', 'method=sample', 'num_samples=10', 'num_warmup=0', 'algorithm=hmc', 'adapt', 'engaged=1']
 retcodes=[3221225785]
 per-chain output files (showing chain 1 only):
 csv_file:
	C:\Users\manny\AppData\Local\Temp\tmpc5xbustq\binomial-rng62poy055\binomial-rng-20230605124511.csv
 console_msgs (if any):
	C:\Users\manny\AppData\Local\Temp\tmpc5xbustq\binomial-rng62poy055\binomial-rng-20230605124511_0-stdout.txt
Consider re-running with show_console=True if the above output is unclear!

In [7]:
import cmdstanpy; cmdstanpy.install_cmdstan(compiler=True)

Installing CmdStan version: 2.32.2
Install directory: C:\Users\manny\.cmdstan
Download successful, file: C:\Users\manny\AppData\Local\Temp\tmpqpbclddh
Extracting distribution
Unpacked download as cmdstan-2.32.2
Building version cmdstan-2.32.2, may take several minutes, depending on your system.
Test model compilation
Installed cmdstan-2.32.2


True